In [ ]:
import os
import pandas as pd
import numpy as np
import cv2

In [ ]:
DATA_ROOT = '../data/'

df_train = pd.read_csv(os.path.join(DATA_ROOT,'train_proc_v2.csv'),index_col='id')
dist = []
mask_pixels = []
masks = []
for id in df_train.index:
    img = cv2.imread(os.path.join('../data/train/images/','{}.png'.format(id)), cv2.IMREAD_GRAYSCALE)
    mask = cv2.imread(os.path.join('../data/train/masks/','{}.png'.format(id)), cv2.IMREAD_GRAYSCALE)
    dist.append(np.unique(img).shape[0])
    mask_pixels.append((mask/255).sum())
    masks.append(np.array(mask)/255.)
df_train['unique_pixels'] = dist
df_train['mask_pixels'] = mask_pixels
df_train['masks'] = masks
df_train[['fold','unique_pixels','coverage_class']].to_csv(os.path.join(DATA_ROOT,'train_proc_v2_gr.csv'))

In [ ]:
DATA_ROOT = '../data/'

df_train = pd.read_csv(os.path.join(DATA_ROOT,'train.csv'),index_col='id')
depths = pd.read_csv(os.path.join(DATA_ROOT,'depths.csv'),index_col='id')
df_train = df_train.join(depths)

# Label suspicious images
dist = []
mask_pixels = []
masks = []
for id in df_train.index:
    img = cv2.imread(os.path.join('../data/train/images/','{}.png'.format(id)), cv2.IMREAD_GRAYSCALE)
    mask = cv2.imread(os.path.join('../data/train/masks/','{}.png'.format(id)), cv2.IMREAD_GRAYSCALE)
    dist.append(np.unique(img).shape[0])
    mask_pixels.append((mask/255).sum())
    masks.append(np.array(mask)/255.)
df_train['unique_pixels'] = dist
df_train['mask_pixels'] = mask_pixels
df_train['masks'] = masks

In [ ]:
df_train.head()

In [ ]:
pd.crosstab(df_train.unique_pixels == 1,pd.isnull(df_train.rle_mask))

In [ ]:
pd.crosstab(df_train.mask_pixels == 0,pd.isnull(df_train.rle_mask))

In [ ]:
df_train = df_train[df_train.unique_pixels > 1].sort_values(['mask_pixels','z'])

In [ ]:
n_folds = 10
df_train['fold'] = (list(range(n_folds)) * df_train.shape[0])[:df_train.shape[0]]

In [ ]:
df_train['fold'].value_counts()

In [ ]:
df_train.head(20)

In [ ]:
#### Reference  from Heng's discussion
# https://www.kaggle.com/c/tgs-salt-identification-challenge/discussion/63984#382657
def get_mask_type(mask):
    border = 10
    outer = np.zeros((101-2*border, 101-2*border), np.float32)
    outer = cv2.copyMakeBorder(outer, border, border, border, border, borderType = cv2.BORDER_CONSTANT, value = 1)

    cover = (mask>0.5).sum()
    if cover == 0:
        return 0 # empty
    if cover == ((mask*outer) > 0.5).sum():
        return 1 #border
    if np.all(mask==mask[0]):
        return 2 #vertical

    percentage = cover/(101*101)
    if percentage < 0.15:
        return 3
    elif percentage < 0.25:
        return 4
    elif percentage < 0.50:
        return 5
    elif percentage < 0.75:
        return 6
    else:
        return 7

def histcoverage(coverage):
    histall = np.zeros((1,8))
    for c in coverage:
        histall[0,c] += 1
    return histall

df_train["coverage"] = df_train.masks.map(np.sum) / pow(101, 2)

df_train["coverage_class"] = df_train.masks.map(get_mask_type)

In [ ]:
df_train.head()

In [ ]:
df_train.coverage_class.value_counts()

In [ ]:
df_train[['fold','coverage_class']].to_csv(os.path.join(DATA_ROOT,'train_proc_v4.csv'))